# 🧠 Cortex-7: Granular Research Lab

Bienvenido a la versión modular de Cortex. Hemos fragmentado el código en **20 pasos ejecutables** para que puedas inspeccionar cada tornillo de la máquina.

---

### 1. Librerías y Setup Inicial

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import random
import requests
from IPython.display import clear_output, display

# Configuración del dispositivo
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🚀 Estamos usando: {device.upper()}")

# Para reproducibilidad (Semilla de Dios)
def set_seed(seed=1337):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
    print(f"🔒 Semilla fijada en: {seed}")

set_seed(1337)

### 2. Carga de Datos (Shakespeare)

In [ ]:
def get_shakespeare():
    url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
    print("📥 Descargando Shakespeare...")
    return requests.get(url).text

text = get_shakespeare()
print(f"Longitud del dataset: {len(text)} caracteres")
print("\n--- Muestra ---")
print(text[:200])

### 3. Tokenización (Byte-Level)
En lugar de un vocabulario complejo, usamos los 256 bytes crudos. Esto hace al modelo universal.

In [ ]:
# Mapeo directo de caracteres a enteros (ASCII/UTF-8)
chars = sorted(list(set(text)))
vocab_size = 256 # Forzamos 256 para ser Byte-Level real, aunque Shakespeare use menos

stoi = { ch:ord(ch) for ch in chars }
itos = { i:chr(i) for i in range(256) }

encode = lambda s: [ord(c) for c in s]
decode = lambda l: ''.join([chr(i) for i in l])

print(f"Tamaño del Vocabulario: {vocab_size} (Bytes)")
test_str = "Hello Cortex"
print(f"Prueba: '{test_str}' -> {encode(test_str)}")

### 4. Preparación de Batches

In [ ]:
data_tensor = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data_tensor))
train_data = data_tensor[:n]
val_data = data_tensor[n:]

def get_batch(split='train', batch_size=32, block_size=64):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix]).to(device)
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]).to(device)
    return x, y

xb, yb = get_batch()
print(f"Batch Shape: {xb.shape}")

### 5. Componente A: Mamba Block (Memoria)
Este bloque reemplaza a la atención para memoria de largo plazo.

In [ ]:
class MambaBlock(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.in_proj = nn.Linear(d_model, d_model * 2)
        self.out_proj = nn.Linear(d_model, d_model)
        self.conv = nn.Conv1d(d_model, d_model, kernel_size=3, padding=1, groups=d_model)
        
    def forward(self, x):
        B, L, D = x.shape
        x_and_res = self.in_proj(x)
        x_val, res = x_and_res.chunk(2, dim=-1)
        x_val = x_val.transpose(1, 2)
        x_val = self.conv(x_val)
        x_val = x_val.transpose(1, 2)
        x_val = F.silu(x_val)
        return self.out_proj(x_val * F.sigmoid(res))

# --- PRUEBA UNITARIA MAMBA ---
dummy_x = torch.randn(1, 64, 128).to(device)
mamba = MambaBlock(128).to(device)
out = mamba(dummy_x)
print(f"Mamba Test: Input {dummy_x.shape} -> Output {out.shape} (Debe ser igual)")

### 6. Componente B: Attention (Razonamiento)
Usamos atención estándar pero instrumentada para ver los mapas de calor.

In [ ]:
class InstrumentedAttention(nn.Module):
    def __init__(self, d_model, n_head):
        super().__init__()
        self.mha = nn.MultiheadAttention(d_model, n_head, batch_first=True)
        self.last_weights = None
        
    def forward(self, x):
        out, weights = self.mha(x, x, x, need_weights=True, average_attn_weights=True)
        self.last_weights = weights.detach().cpu()
        return out

# --- PRUEBA UNITARIA ATENCIÓN ---
attn = InstrumentedAttention(128, 4).to(device)
out = attn(dummy_x)
print(f"Attention Test: Output {out.shape}")

# Visualizar Heatmap Dummy
plt.figure(figsize=(4, 3))
sns.heatmap(attn.last_weights[0].numpy(), cmap='viridis')
plt.title("Prueba de Heatmap (Ruido)")
plt.show()

### 7. El Organismo (Cortex)
Ensamblamos las piezas. Incluimos telemetría para el "Glass Box".

In [ ]:
class CortexOrganism(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.embedding = nn.Embedding(256, config['d_model'])
        self.layers = nn.ModuleList()
        self.activations = {}
        self.residual_velocities = []

        for i in range(config['n_layers']): 
            # Lógica Híbrida: Pares = Mamba, Impares = Atención
            if i % 2 == 0: 
                self.layers.append(MambaBlock(config['d_model']))
            else: 
                self.layers.append(InstrumentedAttention(config['d_model'], config['n_heads']))
                
        self.ln_f = nn.LayerNorm(config['d_model'])
        self.head = nn.Linear(config['d_model'], 256)

    def forward(self, idx, targets=None):
        self.activations = {}
        self.residual_velocities = []
        x = self.embedding(idx)
        
        for i, layer in enumerate(self.layers):
            prev_x = x
            x = layer(x)
            # Telemetría
            with torch.no_grad():
                self.activations[f"layer_{i}"] = x.detach().cpu()
                self.residual_velocities.append((x - prev_x).norm().item())
                
        x = self.ln_f(x)
        logits = self.head(x)
        
        loss = None
        if targets is not None:
            B, T, C = logits.shape
            loss = F.cross_entropy(logits.view(B*T, C), targets.view(B*T))
        return logits, loss

### 8. Generador de Texto
Función auxiliar para ver qué dice el modelo.

In [ ]:
def generate(model, prompt, max_len=50):
    model.eval()
    idx = torch.tensor(encode(prompt), dtype=torch.long).unsqueeze(0).to(device)
    for _ in range(max_len):
        with torch.no_grad():
            logits, _ = model(idx)
            probs = F.softmax(logits[:, -1, :], dim=-1)
            next_token = torch.multinomial(probs, 1)
            idx = torch.cat((idx, next_token), dim=1)
    return decode(idx[0].tolist())

### 9. PRUEBA DE FUEGO: Sanity Check
Entrenamos con UNA sola frase. Si falla aquí, no seguimos.

In [ ]:
print("🏥 Iniciando Sanity Check...")
sanity_text = "To be or not to be"
data = torch.tensor(encode(sanity_text), dtype=torch.long).unsqueeze(0).to(device)
x_sanity, y_sanity = data[:, :-1], data[:, 1:]

model = CortexOrganism({'n_layers': 2, 'd_model': 128, 'n_heads': 4}).to(device)
optim = torch.optim.AdamW(model.parameters(), lr=0.01)

losses = []
for i in range(100):
    _, loss = model(x_sanity, y_sanity)
    optim.zero_grad()
    loss.backward()
    optim.step()
    losses.append(loss.item())
    
    if i % 20 == 0:
        print(f"Iter {i}: Loss {loss.item():.4f} -> '{generate(model, 'To ', 10)}'")

plt.plot(losses)
plt.title("Curva de Aprendizaje (Sanity Check)")
plt.show()

### 10. Glass Box: Atlas de Activación
Visualizamos las neuronas del modelo 'Sanity' que acabamos de entrenar.

In [ ]:
# Pasamos el texto una vez más para capturar activaciones
model(data)
act = model.activations['layer_0'].squeeze(0).numpy()[:, :50].T

plt.figure(figsize=(10, 5))
sns.heatmap(act, cmap='magma')
plt.title("Actividad Neuronal (Capa 0)")
plt.xlabel("Caracteres")
plt.ylabel("Neurona")
plt.xticks(ticks=np.arange(len(sanity_text)), labels=list(sanity_text))
plt.show()

### 11. El Gran Torneo (Configuración)
Definimos los gladiadores.

In [ ]:
configs = [
    {'name': 'Tiny Mamba', 'n_layers': 2, 'd_model': 64, 'n_heads': 2},
    {'name': 'Medium Hybrid', 'n_layers': 4, 'd_model': 128, 'n_heads': 4},
    {'name': 'Big Transformer', 'n_layers': 4, 'd_model': 256, 'n_heads': 8}
]

### 12. Ejecución del Torneo

In [ ]:
results = []
print("⚔️ ¡Comienza el Torneo!")

for cfg in configs:
    print(f"\nEntrenando: {cfg['name']}...")
    m = CortexOrganism(cfg).to(device)
    opt = torch.optim.AdamW(m.parameters(), lr=1e-3)
    
    losses = []
    for i in range(200): # Sprint corto
        xb, yb = get_batch()
        _, loss = m(xb, yb)
        opt.zero_grad()
        loss.backward()
        opt.step()
        losses.append(loss.item())
    
    sample = generate(m, "The ", 30).replace('\n', ' ')
    print(f"   Final Loss: {losses[-1]:.4f}")
    print(f"   Muestra: \"{sample}...\"")
    results.append({'name': cfg['name'], 'loss': losses[-1], 'sample': sample})

### 13. Tabla de Resultados

In [ ]:
df = pd.DataFrame(results).sort_values('loss')
display(df)